In [7]:
%%writefile query1.hql

USE demodb;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(year string,
month string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*'
)
STORED AS TEXTFILE
LOCATION '/data/stackexchange1000/posts/';

Overwriting query1.hql


In [8]:
%%writefile query2.hql

set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.dynamic.partition=true;
set hive.exec.max.dynamic.partitions=2000;
set hive.exec.max.dynamic.partitions.pernode=1000;

USE demodb;

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
GROUP BY year, month;

Overwriting query2.hql


In [9]:
%%writefile query3.hql

USE demodb;

SELECT year, month, count FROM (
    SELECT * FROM (
        SELECT
            count,
            year,
            month,
            ROW_NUMBER() OVER(ORDER BY year, month ASC) as rowid
        FROM posts_sample
    ) AS q1
    ORDER BY rowid ASC
) AS q2
WHERE rowid='3';

Overwriting query3.hql


In [10]:
! hive -f query1.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.602 seconds
OK
Time taken: 0.028 seconds
OK
Time taken: 0.082 seconds
OK
Time taken: 0.639 seconds


In [11]:
! hive -f query2.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.14 seconds
OK
Time taken: 0.098 seconds
OK
Time taken: 0.992 seconds
Query ID = jovyan_20180517114848_6ba30eeb-9d42-4f7c-8c04-ac3e2dab7495
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526488425337_0010, Tracking URL = http://0f7788bc9909:8088/proxy/application_1526488425337_0010/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526488425337_0010
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-17 11:48:49,818 Stage-1 map = 0%, 

Partition demodb.posts_sample{year=2010, month=2010-08} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2010, month=2010-09} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2010, month=2010-10} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2010, month=2010-11} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2010, month=2010-12} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2011, month=2011-01} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2011, month=2011-02} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2011, month=2011-03} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2011, month=2011-04} stats: [numFiles=1, numR

MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1  Reduce: 1   Cumulative CPU: 26.31 sec   HDFS Read: 60007258 HDFS Write: 6803 SUCCESS
Total MapReduce CPU Time Spent: 26 seconds 310 msec
OK
Time taken: 70.204 seconds


In [12]:
! hive -f query3.hql 2> out.log

2008	2008-10	73
